# Equal-Weight BSE 200 Index Fund

**Introduction & Library Imports**

The BSE200 is the india's pular stock market index. 

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each BSE200 constituent you should purchase to get an equal-weight version of the index fund.

**Library Imports**

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

# Importing Our List of Stocks
The next thing we need to do is import the constituents of the BSE200.
Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks=pd.read_excel('BSE200.xlsx') 
stocks

,Symbol
0,500325-IB
1,532540-IB
2,500180-IB
3,500209-IB
4,500696-IB
...,...
195,500260-IB
196,532149-IB
197,506395-IB
198,532754-IB


# Acquiring an API Token
Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.
API tokens (and other sensitive information) should be stored in a secrets.py file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

{IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'} save as your secrets.py file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

# Making Our First API Call
Now it's time to structure our API calls to IEX cloud.

We need the following information from the API:

Market capitalization for each stock
Price of each stock

In [4]:
symbol= '500325-IB'
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data=requests.get(api_url).json()
print(data)

{'avgTotalVolume': 692716, 'calculationPrice': 'close', 'change': 33.31, 'changePercent': 0.01416, 'close': 0, 'closeSource': 'facfioil', 'closeTime': None, 'companyName': 'Reliance Industries Ltd.', 'currency': 'INR', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': 'esCol', 'highTime': 1692073112097, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': None, 'iexCloseTime': None, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': None, 'iexOpenTime': None, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': None, 'latestPrice': 2544.8, 'latestSource': 'Close', 'latestTime': 'February 17, 2022', 'latestUpdate': 1689115263121, 'latestVolume': None, 'low': 0, 'lowSource': 'osCel', 'lowTime': 1691115244541, 'marketCap': 17087132045972, 'oddLotDelayedPrice': None, 'oddLotDelaye

# Parsing Our API Call
The API call that we executed in the last code block contains all of the information required to build our equal-weight BSE200 strategy.

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price=data['latestPrice']
market_cap=data['marketCap']
#market_cap/1e12 to calculate market_cap per trillion

# Adding Our Stocks Data to a Pandas DataFrame
The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns=['Ticker','Stock Price','Market Captalization','Number of shares to BUY']
final_datafreame=pd.DataFrame(columns=my_columns)
final_datafreame

,Ticker,Stock Price,Market Captalization,Number of shares to BUY


In [7]:
final_datafreame.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
        
    ],
        index =my_columns
        #tells which columns this data should be added
    ),
    ignore_index=True

)

,Ticker,Stock Price,Market Captalization,Number of shares to BUY
0,500325-IB,2544.8,17087132045972,N/A


# Looping Through The Tickers in Our List of Stocks
Using the same logic that we outlined above, we can pull data for all BSE200 stocks and store their data in the DataFrame using a for loop.

In [8]:
final_dataframe=pd.DataFrame(columns=my_columns)
for stock in stocks['Symbol'][:5]:
    api_url=f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(api_url).json()
    final_dataframe=final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
            index=my_columns
        ),
        ignore_index=True
    )

In [9]:
final_dataframe

,Ticker,Stock Price,Market Captalization,Number of shares to BUY
0,500325-IB,2538.20,16627491919606,N/A
1,532540-IB,3888.07,14386621530393,N/A
2,500180-IB,1550.50,8544719530737,N/A
3,500209-IB,1754.80,7437348154305,N/A
4,500696-IB,2413.40,5426295985755,N/A


# Using Batch API Calls to Improve Performance
Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 200 to 2 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [10]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [11]:
symbol_groups = list(chunks(stocks['Symbol'], 50))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series(
                                            [      symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'
                                            ], 
                                                  index = my_columns
                                        ), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Stock Price,Market Captalization,Number of shares to BUY
0,500325-IB,2522.90,16748312120177,N/A
1,532540-IB,3897.38,14366876238353,N/A
2,500180-IB,1517.40,8488685969763,N/A
3,500209-IB,1740.61,7289498023146,N/A
4,500696-IB,2328.00,5497664435562,N/A
...,...,...,...,...
195,500260-IB,894.68,207425317624,N/A
196,532149-IB,54.70,220613467431,N/A
197,506395-IB,818.80,235391108284,N/A
198,532754-IB,42.20,246247486314,N/A


# Calculating the Number of Shares to Buy
As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [12]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:100000000


In [16]:
position_size = val/ len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i,'Number of shares to BUY']=math.floor(position_size/final_dataframe.loc[i,'Stock Price'])

final_dataframe

,Ticker,Stock Price,Market Captalization,Number of shares to BUY
0,500325-IB,2522.90,16748312120177,198
1,532540-IB,3897.38,14366876238353,128
2,500180-IB,1517.40,8488685969763,329
3,500209-IB,1740.61,7289498023146,287
4,500696-IB,2328.00,5497664435562,214
...,...,...,...,...
195,500260-IB,894.68,207425317624,558
196,532149-IB,54.70,220613467431,9140
197,506395-IB,818.80,235391108284,610
198,532754-IB,42.20,246247486314,11848


# Formatting Our Excel Output
We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

**Initializing our XlsxWriter Object**

In [22]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

# **Creating the Formats We'll Need For Our .xlsx File**

Formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:


//String format for tickers

//INR XX.XX format for stock prices

//INR XX,XXX format for market capitalization

//Integer format for the number of shares to purchase
 
    

In [23]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

inr_format = writer.book.add_format(
        {
            'num_format':'INR0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

Applying the Formats to the Columns of Our .xlsx File
We can use the set_column method applied to the writer.sheets['Recommended Trades'] object to apply formats to specific columns of our spreadsheets.

Here's an example:

writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_format #This applies the format 'string_format' to the column
                    )

In [25]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', inr_format],
                    'C': ['Market Capitalization', inr_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

**Saving our Excel file is very easy:**

In [26]:
writer.save()